**Task 08: Completing missing data**

In [ ]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

In [ ]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g1.parse(github_storage+"resources/data01.rdf", format="xml")
g2.parse(github_storage+"resources/data02.rdf", format="xml")

Tarea: lista todos los elementos de la clase Person en el primer grafo (data01.rdf) y completa los campos (given name, family name y email) que puedan faltar con los datos del segundo grafo (data02.rdf). Puedes usar consultas SPARQL o iterar el grafo, o ambas cosas.

**============= SOLUCIÓN PROPUESTA =============**

In [ ]:
from rdflib import XSD
from rdflib.plugins.sparql import prepareQuery

VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
NS = Namespace("http://data.org#")

# Obtenemos los sujetos y sus propiedades (existan o no)

props = [
         VCARD.EMAIL, VCARD.Given, VCARD.Family
]

q1 = prepareQuery('''
  SELECT 
    ?subject ?email ?given ?family
	WHERE { 
    ?subject a :Person.
  
    OPTIONAL{
      ?subject vcard:EMAIL ?email.
    }
    OPTIONAL{
      ?subject vcard:Given ?given.
    }
    OPTIONAL{
      ?subject vcard:Family ?family.
    }
  }
  ''',
  initNs = { "vcard": VCARD, "": NS}
)

# Obtener una propiedad determinada del otro grafo

q2 = prepareQuery('''

  SELECT
    ?o
  WHERE {
    ?s a :Person.
    ?s ?p ?o.
  }

  ''',
  initNs = { "vcard": VCARD, "": NS}
)

# Insertar una propiedad nueva en el grafo

q3 = ('''

  INSERT{
    ?s ?p ?o
  }
  WHERE{
    ?s a :Person
  }

  '''
)

for r in g1.query(q1):

  # Comprobamos si para un sujeto dado, le falta alguna propiedad y añadimos lo que falte

  if r[1] == None:

    r2 = g2.query(q2, initBindings ={'?s': r[0], '?p': props[0]})

    binds = {'?s': r[0], '?p': props[0], '?o': list(r2)[0][0]}

    g1.update(q3, initBindings = binds, initNs = { "vcard": VCARD, "": NS})

  if r[2] == None:

    r2 = g2.query(q2, initBindings ={'?s': r[0], '?p': props[1]})

    binds = {'?s': r[0], '?p': props[1], '?o': list(r2)[0][0]}

    g1.update(q3, initBindings = binds, initNs = { "vcard": VCARD, "": NS})  
    
  if r[3] == None:

    r2 = g2.query(q2, initBindings ={'?s': r[0], '?p': props[2]})

    binds = {'?s': r[0], '?p': props[2], '?o': list(r2)[0][0]}

    g1.update(q3, initBindings = binds, initNs = { "vcard": VCARD, "": NS})  
    


Comprobamos que para las 4 personas estén las 3 propiedades completas

In [ ]:
for t in g1.triples((None, VCARD.EMAIL, None)):
  print(t)
print("----")
for t in g1.triples((None, VCARD.Given, None)):
  print(t)
print("----")
for t in g1.triples((None, VCARD.Family, None)):
  print(t)
